In [ ]:
import pandas as pd
import numpy as np
import joblib
import json

✅ Models Loaded


In [8]:
iso_model = joblib.load("isolation_forest_model.pkl")
rf_model = joblib.load("random_forest_model.pkl")

print("Models Loaded Successfully")

Models Loaded Successfully


In [2]:
FEATURES = [
    "mean_cpu",
    "std_cpu",
    "min_cpu",
    "max_cpu",
    "delta_cpu",
    "cpu_trend",
    "cpu_volatility",
    "mean_memory",
    "std_memory",
    "memory_trend",
    "mean_requests",
    "request_spike_count",
    "throughput_delta",
    "cost_delta",
    "unit_economics_ratio"
]

In [10]:
test_df = pd.read_csv(r"C:\Users\Athul VR\OneDrive\Desktop\ML Model\featuring\randomdata_final.csv")

print("Test Data Shape:", test_df.shape)
test_df.head()

Test Data Shape: (1050, 16)


,mean_cpu,std_cpu,min_cpu,max_cpu,delta_cpu,cpu_trend,cpu_volatility,mean_memory,std_memory,memory_trend,mean_requests,request_spike_count,throughput_delta,cost_delta,unit_economics_ratio,incident_type
0,50,5,45,59,3,0.14,0.11,46,4,0.11,1107,0,46,5,1.40,Normal
1,54,3,46,66,6,0.23,0.14,49,2,0.16,1140,0,50,6,1.33,Normal
2,85,13,58,95,33,3.02,1.94,56,6,0.34,1531,3,110,47,0.46,BadDeploy
3,88,18,53,95,38,2.27,1.66,57,4,0.46,1597,3,134,50,0.57,BadDeploy
4,52,4,44,66,4,0.31,0.17,46,4,0.15,1236,0,45,7,1.38,Normal


In [12]:
X_test = test_df[FEATURES].copy()

In [13]:
iso_raw_score = iso_model.decision_function(X_test)
iso_pred = iso_model.predict(X_test)

# Convert to Risk %
risk_score = (iso_raw_score.max() - iso_raw_score) / (
    iso_raw_score.max() - iso_raw_score.min() + 1e-6
)

risk_score = np.clip(risk_score, 0, 1)

test_df["anomaly_score"] = risk_score
test_df["is_anomaly"] = np.where(iso_pred == -1, True, False)

In [14]:
rf_probs = rf_model.predict_proba(X_test)
rf_pred = rf_model.predict(X_test)

rf_confidence = np.max(rf_probs, axis=1)

test_df["incident_type"] = rf_pred
test_df["confidence"] = rf_confidence

# If NOT anomaly → force Normal
test_df.loc[test_df["is_anomaly"] == False, "incident_type"] = "Normal"

In [15]:
def generate_indicators(row):
    indicators = []

    if row["mean_cpu"] > 80:
        indicators.append(f"High CPU ({row['mean_cpu']:.1f}%)")

    if row["cpu_volatility"] > 0.5:
        indicators.append("Unstable CPU pattern")

    if row["memory_trend"] > 1:
        indicators.append("Memory increasing trend")

    if row["request_spike_count"] > 5:
        indicators.append("Traffic spikes detected")

    if len(indicators) == 0:
        indicators.append("System stable")

    return indicators

test_df["indicators"] = test_df.apply(generate_indicators, axis=1)

In [18]:
def build_backend_output(df, n=20):
    predictions = []

    for _, row in df.head(n).iterrows():
        pred = {
            "anomaly_score": float(row["anomaly_score"]),
            "is_anomaly": bool(row["is_anomaly"]),
            "incident_type": str(row["incident_type"]),
            "confidence": float(row["confidence"]),
            "indicators": row["indicators"],
            "features": {f: float(row[f]) for f in FEATURES}
        }

        predictions.append(pred)

    return {"predictions": predictions}

In [21]:
import json
backend_output = build_backend_output(test_df, n=20)

print(json.dumps(backend_output, indent=2))

{
  "predictions": [
    {
      "anomaly_score": 0.2973904958022674,
      "is_anomaly": false,
      "incident_type": "Normal",
      "confidence": 1.0,
      "indicators": [
        "System stable"
      ],
      "features": {
        "mean_cpu": 50.0,
        "std_cpu": 5.0,
        "min_cpu": 45.0,
        "max_cpu": 59.0,
        "delta_cpu": 3.0,
        "cpu_trend": 0.14,
        "cpu_volatility": 0.11,
        "mean_memory": 46.0,
        "std_memory": 4.0,
        "memory_trend": 0.11,
        "mean_requests": 1107.0,
        "request_spike_count": 0.0,
        "throughput_delta": 46.0,
        "cost_delta": 5.0,
        "unit_economics_ratio": 1.4
      }
    },
    {
      "anomaly_score": 0.18208377233675876,
      "is_anomaly": false,
      "incident_type": "Normal",
      "confidence": 1.0,
      "indicators": [
        "System stable"
      ],
      "features": {
        "mean_cpu": 54.0,
        "std_cpu": 3.0,
        "min_cpu": 46.0,
        "max_cpu": 66.0,
        

In [22]:
with open("backend_predictions.json", "w") as f:
    json.dump(backend_output, f, indent=2)

print("Backend JSON Saved")

Backend JSON Saved
